In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import ast

In [90]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, RepeatVector, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import backend as K
import generate_valid_prediction
from keras.models import save_model, load_model

In [91]:
note_mapping = {
    'C0': 1, 'C#0': 2, 'D0': 3, 'D#0': 4, 'E0': 5, 'F0': 6, 'F#0': 7, 'G0': 8, 'G#0': 9, 'A1': 10,
    'A#1': 11, 'B1': 12, 'C1': 13, 'C#1': 14, 'D1': 15, 'D#1': 16, 'E1': 17, 'F1': 18, 'F#1': 19,
    'G1': 20, 'G#1': 21, 'A2': 22, 'A#2': 23, 'B2': 24, 'C2': 25, 'C#2': 26, 'D2': 27, 'D#2': 28,
    'E2': 29, 'F2': 30, 'F#2': 31, 'G2': 32, 'G#2': 33, 'A3': 34, 'A#3': 35, 'B3': 36, 'C3': 37,
    'C#3': 38, 'D3': 39, 'D#3': 40, 'E3': 41, 'F3': 42, 'F#3': 43, 'G3': 44, 'G#3': 45, 'A4': 46,
    'A#4': 47, 'B4': 48, 'C4': 49, 'C#4': 50, 'D4': 51, 'D#4': 52, 'E4': 53, 'F4': 54, 'F#4': 55,
    'G4': 56, 'G#4': 57, 'A5': 58, 'A#5': 59, 'B5': 60, 'C5': 61, 'C#5': 62, 'D5': 63, 'D#5': 64,
    'E5': 65, 'F5': 66, 'F#5': 67, 'G5': 68, 'G#5': 69, 'A6': 70, 'A#6': 71, 'B6': 72, 'C6': 73
}

In [92]:
dataset_1 = [
    (['C1', 'G1', 'A1', 'F1'], (['C1', 'E1', 'G1'], ['G1', 'B2', 'D2'], ['A1', 'C1', 'E1'], ['F1', 'A2', 'C2'])),
    (['C4', 'F4', 'G4', 'A5'], (['C4', 'E4', 'G4'], ['F4', 'C5', 'A5'], ['G4', 'B5', 'D5'], ['A5', 'C5', 'A6'])),
    (['C2', 'D2', 'F2', 'G2'], (['C2', 'G2', 'E3'], ['D2', 'A3', 'F2', 'D3'], ['F2', 'A3', 'C3'], ['G2', 'B3', 'D3']))
]

In [93]:
def encode_notes(notes):
    return [note_mapping[note] for note in notes]

In [94]:
dataset = pd.read_csv("chord_output.csv")

In [132]:
# Process the dataset
X_train = []
y_train = []
X_test = []
y_test = []

#-----------------------------------------------
# for base_notes, chord_progressions in dataset_1:
#     # Encode base notes
#     encoded_base_notes = encode_notes(base_notes)

#     # print(base_notes)

#     # Encode and pad chord progressions
#     encoded_chord_progressions = [encode_notes(chord) for chord in chord_progressions]
#     print(type(encoded_chord_progressions))
#     padded_chord_progressions = pad_sequences(encoded_chord_progressions, maxlen=6, padding='post')

#     # Create input-output pairs
#     X_train.append(encoded_base_notes)
#     y_train.append(padded_chord_progressions)

# X_train = np.array(X_train)
# y_train = np.array(y_train)

# # print("Here's X:")
# # print(X_train)
# # print("Here's y:")
# # print(y_train)
#----------------------------------------------

# # Split the data into training and validation sets
# train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# # Prepare training data
# X_train = np.array(train_data['base_notes'].tolist())
# y_train = np.array(train_data['chord_progressions'].tolist())

# # Convert string representations to lists
# X_train = [ast.literal_eval(x) for x in X_train]
# y_train = [ast.literal_eval(y) for y in y_train]

# X_train = np.array(X_train)
# y_train = np.array(y_train)

num_lines = 83
test_amt = 16

for i in range(num_lines - test_amt):
    base_note = ast.literal_eval(dataset["base_notes"][i])
    chord_progression_raw = ast.literal_eval(dataset["chord_progressions"][i])

    chord_progression = pad_sequences(chord_progression_raw, maxlen=4, padding='post')
    
    X_train.append(base_note)
    y_train.append(chord_progression)

# print(np.array(dataset["base_notes"][99]))

X_train = np.array(X_train)
y_train = np.array(y_train)


for i in range(num_lines - test_amt, num_lines):
    base_note = ast.literal_eval(dataset["base_notes"][i])
    chord_progression_raw = ast.literal_eval(dataset["chord_progressions"][i])

    chord_progression = pad_sequences(chord_progression_raw, maxlen=4, padding='post')
    
    X_test.append(base_note)
    y_test.append(chord_progression)

# print(np.array(dataset["base_notes"][99]))

X_test = np.array(X_test)
y_test = np.array(y_test)

print("Here's X:")
print(X_train)
print("Here's y:")
print(y_train)

Here's X:
[[1 5 4 6]
 [1 5 4 6]
 [1 5 4 6]
 [1 5 6 4]
 [1 5 6 4]
 [1 5 6 4]
 [1 6 4 5]
 [1 6 4 5]
 [1 6 4 5]
 [1 6 5 4]
 [1 6 5 4]
 [1 6 5 4]
 [1 6 5 4]
 [1 6 5 4]
 [2 4 6 5]
 [2 4 6 5]
 [3 5 6 1]
 [3 5 6 1]
 [3 5 6 1]
 [3 5 6 1]
 [3 5 6 3]
 [3 5 6 3]
 [4 1 5 3]
 [4 1 5 4]
 [4 1 5 4]
 [4 1 5 4]
 [4 1 5 4]
 [4 1 6 5]
 [4 1 6 5]
 [4 1 6 5]
 [4 1 6 5]
 [4 1 6 5]
 [4 3 6 5]
 [4 3 6 5]
 [4 3 6 5]
 [4 5 6 1]
 [4 5 6 1]
 [4 5 6 1]
 [4 5 6 1]
 [4 6 5 3]
 [4 6 5 3]
 [6 1 2 4]
 [6 1 2 4]
 [6 1 2 4]
 [6 1 3 4]
 [6 1 3 4]
 [6 1 3 4]
 [6 1 4 5]
 [6 1 4 5]
 [6 1 4 5]
 [6 1 5 2]
 [6 1 5 2]
 [6 1 5 4]
 [6 1 5 4]
 [6 1 5 4]
 [6 1 5 4]
 [6 1 6 5]
 [6 1 6 5]
 [6 1 6 5]
 [6 2 4 5]
 [6 2 4 5]
 [6 2 4 5]
 [6 2 4 5]
 [6 2 5 1]
 [6 2 5 1]
 [6 2 5 1]
 [6 2 5 1]]
Here's y:
[[[26 24 22 15]
  [30 21 26 19]
  [29 20 25 18]
  [31 29 20 13]]

 [[33 31 29 15]
  [33 30 21 19]
  [27 32 29 18]
  [27 31 29 13]]

 [[33 31 29 15]
  [30 21 26 19]
  [32 29 20 18]
  [31 29 20 13]]

 ...

 [[27 31 29 13]
  [32 30 23 20]
  [33 

In [133]:
def custom_loss(y_true, y_pred):
    # Your existing loss term
    mse_loss = K.mean(K.square(y_true - y_pred))
    
     # Additional term penalizing rule violations
    rule_penalty = calculate_rule_penalty(y_pred)
    
    # Combine the losses
    total_loss = mse_loss + 0.1 * rule_penalty  # Adjust the weight according to your preference
    
    return total_loss

def calculate_rule_penalty(chord_progression):
    print(chord_progression)
    return 5

In [140]:
# model = Sequential()
# model.add(Dense(units=128, activation = "relu"))
# model.add(Dense(units=4, activation='softmax'))

model = Sequential()
model.add(Dense(units=128, activation="relu", input_shape=(4,)))
model.add(RepeatVector(4))  # Repeat the vector to match the number of timesteps
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(Dense(units=32, activation="relu"))
model.add(TimeDistributed(Dense(units=4, activation='softmax')))

In [141]:

model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), loss=custom_loss, metrics=['accuracy'])

In [142]:
model.fit(X_train, y_train, epochs=100, batch_size=1, validation_split=0.2)

Epoch 1/100


TypeError: in user code:

    File "C:\Users\charl\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\charl\AppData\Local\Temp\ipykernel_12508\1870434609.py", line 3, in custom_loss  *
        mse_loss = K.mean(K.square(y_true - y_pred))

    TypeError: Input 'y' of 'Sub' Op has type float32 that does not match type int32 of argument 'x'.


In [ ]:
model.save("first_version.h5")

In [ ]:
loaded_model = load_model("first_version.h5", custom_objects={'custom_loss': custom_loss})
loaded_model.predict(np.array([[4, 2, 5, 6]]))

In [ ]:
print(np.array([1, 5, 6, 4]))

In [ ]:
def validity_function(base_notes, chord_progression):
    validity_array = []
    base_multiple_array = []
    for i in range(4):
        for j in range(4):
            if ((abs(base_notes[i] - chord_progression[i][j]) % 7 == 0).all() or 
                (abs(base_notes[i] - chord_progression[i][j]) % 7 == 2).all() or 
                (abs(base_notes[i] - chord_progression[i][j]) % 7 == 4).all()):
                validity_array.append(True)
            else:
                validity_array.append(False)
            if (abs(base_notes[i] - chord_progression[i][j]) % 7 == 0).all():
                base_multiple_array.append(True)
            else:
                base_multiple_array.append(False)
    sub1_1 = validity_array[0:4]
    sub1_2 = validity_array[4:8]
    sub1_3 = validity_array[8:12]
    sub1_4 = validity_array[12:16]
    sub2_1 = base_multiple_array[0:4]
    sub2_2 = base_multiple_array[4:8]
    sub2_3 = base_multiple_array[8:12]
    sub2_4 = base_multiple_array[12:16]
    if (False in sub1_1 or False in sub1_2 or False in sub1_3 or False in sub1_4):
        return False
    elif (True in sub2_1 and True in sub2_2 and True in sub2_3 and True in sub2_4):
        return True
    else:
        return False

def generate_valid_prediction(model, input_sequence):
    for i in range(100):
        prediction = model.predict(np.array([input_sequence]))
        print(prediction)
        if validity_function(input_sequence, prediction[0]):
            return prediction
    # while not validity_function(input_sequence, prediction[0]):
    #     # prediction = np.round(model.predict(np.array([input_sequence])) * 125).astype(int)
    #     prediction = loaded_model.predict(np.array([input_sequence]))
    #     if validity_function(input_sequence, prediction[0]):
    #         return prediction

In [ ]:
print(generate_valid_prediction(model, [1, 4, 5, 6]))
# model.predict(np.array([[1, 4, 5, 6]]))

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

In [ ]:
np.round(loaded_model.predict(np.array([[4, 2, 5, 6]])) * 125).astype(int)